## 导入包


In [1]:
# 基础的数据处理工具
import numpy as np
import pandas as pd

# 可视化
import matplotlib.pyplot as plt

# 处理python时间函数
import datetime

# 处理nc数据
import netCDF4 as nc
from netCDF4 import num2date

# 处理网格数据，shp之类的
import geopandas as gpd

# 处理tiff文件
import rasterio

# gis的一些逻辑判断
from shapely.geometry import Point

# 设置投影坐标系等
from cartopy import crs as ccrs

# 打印进度条
from tqdm import tqdm

tqdm.pandas()

# 并行

from joblib import Parallel, delayed

# 检测系统

import platform

# matplotlib 显示中文的问题
if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

## 加载数据

In [2]:
shp_data = gpd.read_file("./数据集/Pearl王川/shp-数据1/ca_Union.shp")

nc_1988tp = nc.Dataset("./数据集/Pearl王川/1988tp.nc")

In [3]:
for item in nc_1988tp.variables.values():
    print('*' * 70)
    print(item)

**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
int64 time(time)
    units: days since 1988-01-01 00:00:00
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (366,)
filling on, default _FillValue of -9223372036854775806 used
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    _FillValue: nan
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (611,)
filling on
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    _FillValue: nan
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (221,)
filling on
**********************************************************************
<class 'netCDF4._netCDF4.Variable'>
float32 tp(time, latitude, longitude)
    _FillValue: nan
unlimit

<ipython-input-3-e3e8d12b5fc0>:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(item)


In [4]:
raw_longitude = np.array(nc_1988tp.variables['longitude'])
raw_latitude = np.array(nc_1988tp.variables['latitude'])
raw_time = np.array(nc_1988tp.variables['time'])
raw_tp = np.array(nc_1988tp.variables['tp'])

<ipython-input-4-3db7c478fb2a>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_longitude = np.array(nc_1988tp.variables['longitude'])
<ipython-input-4-3db7c478fb2a>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_latitude = np.array(nc_1988tp.variables['latitude'])
<ipython-input-4-3db7c478fb2a>:3: DeprecationWarning: `np.bool` is a depre

## 插值

In [16]:
# [ 89.75 -89.75]
# [-179.75 179.75]
target_lon = -179.75 + 0.5 * np.arange(0, 720)
target_lat = -89.75 + 0.5 * np.arange(0, 360)

target_value = []

from scipy.interpolate import interp2d

for i in tqdm(range(raw_time.shape[0])):
    f = interp2d(raw_longitude, raw_latitude, raw_tp[i, :, :])
    temp_value = f(target_lon, target_lat)
    target_value.append(temp_value)

target_value = np.array(target_value)
target_value.shape

100%|██████████| 366/366 [00:02<00:00, 140.48it/s]


(366, 360, 720)

## 写入nc文件

In [21]:
with nc.Dataset('./结果/test1115.nc',mode='w',format='NETCDF4_CLASSIC') as ncfile:

    # 创建维度
    lat_dim = ncfile.createDimension('lat', 360)     # latitude axis
    lon_dim = ncfile.createDimension('lon', 720)    # longitude axis
    time_dim = ncfile.createDimension('time', None)

    # 创建变量
    lat = ncfile.createVariable('lat', np.float32, ('lat',))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'

    lon = ncfile.createVariable('lon', np.float32, ('lon',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'

    time = ncfile.createVariable('time', np.float64, ('time',))
    time.units = 'days since 1988-01-01 00:00:00'
    time.long_name = 'time'

    temp = ncfile.createVariable('temp',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost
    # temp.units = 'K' # degrees Kelvin
    temp.standard_name = 'air_temperature'


    # 写入变量
    lat[:] = target_lat
    lon[:] = target_lon
    time[:] = raw_time
    temp[:, :, :] = target_value

### 测试

In [24]:
import xarray as xr
test_xr = xr.open_dataset("./结果/test1115.nc")
test_xr

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 366)
Coordinates:
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float32 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * time     (time) datetime64[ns] 1988-01-01 1988-01-02 ... 1988-12-31
Data variables:
    temp     (time, lat, lon) float64 ...

In [43]:
test_nc = nc.Dataset("./结果/test1115.nc")

for item in test_nc.variables.values():
    print('*' * 70)
    print(item.name)
    print(item)

**********************************************************************
lat
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (360,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
lon
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (720,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
time
<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: days since 1988-01-01 00:00:00
    long_name: time
unlimited dimensions: time
current shape = (366,)
filling on, default _FillValue of 9.969209968386869e+36 used
**********************************************************************
temp
<class 'netCDF4._netCDF4.Variable'>
float64 temp(tim

<ipython-input-43-32037f2bb0f0>:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(item)


## 裁切

In [44]:
class GetMask(object):
    def __init__(self,
                 geopandas: gpd.GeoDataFrame,
                 nc_data: nc.Dataset,
                 nc_variable: str,
                 lat_variable: str,
                 lon_variable: str,
                 time_variable: str):
        self.geopandas = geopandas
        self.nc_data = nc_data
        self.nc_variable = nc_variable
        self.lat_variable = lat_variable
        self.lon_variable = lon_variable
        self.time_variable = time_variable
        self.nc_target_data = None
        self.target_data_missing_value = None
        self.time_dim = None
        self.lat_dim = None
        self.lon_dim = None
        self.mask_matrix = None
        self.longitude_data = None
        self.latitude_data = None
        self.time_data = None
        self.time_units = None
        self.clean_time_data = None

    def num2datetime(self, cftime, units, format='%Y-%m-%d %H:%M:%S'):
        """
        将nc文件里面的时间格式 从cftime 转换到 datetime格式
        :param cftime:
        :param units:
        :param format:
        :return:
        """
        return datetime.datetime.strptime(num2date(times=cftime, units=units).strftime(format), format)

    @staticmethod
    def array2gtiff(array, filename):
        """
        将一个矩阵保存为tiff文件,
        这里还可以设置tiff的crs和transofrm。更多，可以查看rasterio的官网或者下面的这个链接
        https://gis.stackexchange.com/questions/279953/numpy-array-to-gtiff-using-rasterio-without-source-raster
        :param array: shape:(row, col)
        :param filename:
        :return:
        """
        with rasterio.open(filename, 'w', driver='GTiff',
                           height=array.shape[0], width=array.shape[1],
                           count=1, dtype=str(array.dtype)) as f:
            f.write(array, 1)

    def pic(self, lon, lat) -> bool:

        """
        检测一个点是否在中国边界线内
        lon:东经
        lat:北纬
        :param lon:
        :param lat:
        :return:
        """
        return self.geopandas.contains(Point(lon, lat))[0]

    def parallel_mask(self, index_lon, index_lat):
        point = (self.longitude_data[index_lon], self.latitude_data[index_lat])
        value = self.pic(lon=point[0], lat=point[1])
        # return value
        self.mask_matrix[index_lat, index_lon] = value

    def run(self):
        # 处理geopandas数据
        # self.geopandas = self.geopandas.iloc[0, :]
        self.geopandas['geometry'] = self.geopandas.buffer(0)

        # 处理nc数据
        self.nc_target_data = np.array(self.nc_data.variables[self.nc_variable])

        if 'missing_value' in dir(self.nc_data.variables[self.nc_variable]):
            self.target_data_missing_value = self.nc_data.variables[self.nc_variable].missing_value
        else:
            self.target_data_missing_value = np.nan

        self.nc_target_data[self.nc_target_data == self.target_data_missing_value] = np.nan

        # 提取lat,lon,lat 变量
        self.longitude_data = np.array(self.nc_data.variables[self.lon_variable])
        self.latitude_data = np.array(self.nc_data.variables[self.lat_variable])
        self.time_units = self.nc_data.variables[self.time_variable].units
        self.time_data = np.array(self.nc_data.variables[self.time_variable])
        self.clean_time_data = [self.num2datetime(cftime=i, units=self.time_units) for i in self.time_data]

        # 创建一个mask
        nc_target_data_shape = self.nc_target_data.shape

        if len(nc_target_data_shape) == 3:
            (self.time_dim, self.lat_dim, self.lon_dim) = nc_target_data_shape
        else:
            (self.lat_dim, self.lon_dim) = nc_target_data_shape

        self.mask_matrix = np.full(shape=(self.lat_dim, self.lon_dim), fill_value=False)

        _ = Parallel(n_jobs=-1, backend='threading', verbose=0)(
            delayed(self.parallel_mask)(index_lon, index_lat)
            for index_lon in tqdm(range(self.lon_dim))
            for index_lat in range(self.lat_dim))


    def getclipdata(self):
        value = self.nc_target_data.copy()
        for i in tqdm(range(self.time_data.shape[0])):
            temp = value[i, :, :]
            temp[~self.mask_matrix] = np.nan
            value[i, :, :] = temp

        return value


test = GetMask(geopandas=shp_data, nc_data=test_nc, nc_variable='temp', lat_variable='lat',
               lon_variable='lon', time_variable='time')

test.run()

<ipython-input-44-1cb7bc6e1b9d>:76: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.nc_target_data = np.array(self.nc_data.variables[self.nc_variable])
<ipython-input-44-1cb7bc6e1b9d>:78: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if 'missing_value' in dir(self.nc_data.variables[self.nc_variable]):
<ipython-input-44-1cb7bc6e1b9d>:86: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: http

In [45]:
clip_test = test.getclipdata()

100%|██████████| 366/366 [00:00<00:00, 3119.49it/s]


## 测试一下这个函数

In [46]:
Lon_data, Lat_data = np.meshgrid(target_lon, target_lat)

plot_data = pd.DataFrame({'lon': Lon_data.flatten(),
                          'lat': Lat_data.flatten(),
                          'mask': test.mask_matrix.flatten()})

In [47]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7, 6), dpi=150)
shp_data.boundary.plot(ax=ax, color='black')
ax.grid()

plot_data_in = plot_data.loc[plot_data['mask']]
ax.scatter(plot_data_in['lon'], plot_data_in['lat'], s=0.1)

plot_data_out = plot_data.loc[~plot_data['mask']]
ax.scatter(plot_data_out['lon'], plot_data_out['lat'], s=0.1, c='red')

In [54]:
%matplotlib
fig, ax = plt.subplots(figsize=(7, 6), dpi=150)

im = ax.imshow(clip_test[1, :, :][::-1,:], cmap=plt.cm.get_cmap('RdYlBu'))

fig.colorbar(im,orientation='vertical')

Using matplotlib backend: MacOSX


In [55]:
with nc.Dataset('./结果/test11152.nc',mode='w',format='NETCDF4_CLASSIC') as ncfile:

    # 创建维度
    lat_dim = ncfile.createDimension('lat', 360)     # latitude axis
    lon_dim = ncfile.createDimension('lon', 720)    # longitude axis
    time_dim = ncfile.createDimension('time', None)

    # 创建变量
    lat = ncfile.createVariable('lat', np.float32, ('lat',))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'

    lon = ncfile.createVariable('lon', np.float32, ('lon',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'

    time = ncfile.createVariable('time', np.float64, ('time',))
    time.units = 'days since 1988-01-01 00:00:00'
    time.long_name = 'time'

    temp = ncfile.createVariable('temp',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost
    # temp.units = 'K' # degrees Kelvin
    temp.standard_name = 'air_temperature'


    # 写入变量
    lat[:] = target_lat
    lon[:] = target_lon
    time[:] = raw_time
    temp[:, :, :] = clip_test